In [1]:
import sys  # Remove in production - KTZ

sys.path.append("..")  # Remove in production - KTZ
import pandas as pd
import datetime
from PolygonTickData.Helper import Helper
from CalculateETFArbitrage.LoadEtfHoldings import LoadHoldingsdata
from CalculateETFArbitrage.GatherData import DataApi
import numpy as np

# Jupyter Notebook to display all the columns in a df
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', -1)  # or 199

In [2]:
def EtfMover(df=None, columnName=None):
    df=round(df,4)
    arr=df.abs().values.argsort(1)[:, -10:][:, ::-1]
    pos=0
    Change1=[]
    Change2=[]
    Change3=[]
    Change4=[]
    Change5=[]
    Change6=[]
    Change7=[]
    Change8=[]
    Change9=[]
    Change10=[]
    for i in arr:
        top=df.iloc[pos,].to_dict()
        l=[(k,v) for k,v in top.items()]
        Change1.append(l[i[0]])
        Change2.append(l[i[1]])
        Change3.append(l[i[2]])
        Change4.append(l[i[3]])
        Change5.append(l[i[4]])
        Change6.append(l[i[5]])
        Change7.append(l[i[6]])
        Change8.append(l[i[7]])
        Change9.append(l[i[8]])
        Change10.append(l[i[9]])
        pos+=1
    result=pd.DataFrame([Change1,Change2,Change3,
                         Change4,Change5,Change6,
                         Change7,Change8,Change9,
                         Change10]).T
    result.index=df.index
    result.columns=[columnName+str(i) for i in range(1,11)]
    return result

In [3]:
etfname = 'XLK'
date = '2020-03-17'

etfData = LoadHoldingsdata().LoadHoldingsAndClean(etfname=etfname, fundholdingsdate=date)
allData = DataApi(etfname=etfname, date=date, etfData=etfData)

Can't Fetch Fund Holdings Data
52.91.26.227:27017: [Errno 61] Connection refused


AttributeError: 'NoneType' object has no attribute 'getSymbols'

In [ ]:
helperObj = Helper()
allData.tradesDataDf['Time'] = allData.tradesDataDf['Time'].apply(lambda x: helperObj.getHumanTime(ts=x, divideby=1000))
tradePricesDFMinutes = allData.tradesDataDf.groupby([allData.tradesDataDf['Time'], allData.tradesDataDf['Symbol']])['Trade Price']
tradePricesDFMinutes = tradePricesDFMinutes.first().unstack(level=1)

priceforNAVfilling = allData.openPriceData.set_index('Symbol').T.to_dict('records')[0]

In [ ]:
tradePricesDFMinutes

In [ ]:
marketStartTime = datetime.datetime.strptime('13:29:00', "%H:%M:%S").time()
marketEndTime = datetime.datetime.strptime('20:00:00', "%H:%M:%S").time()

mask = (tradePricesDFMinutes.index.time >= marketStartTime) & (tradePricesDFMinutes.index.time <= marketEndTime)
tradePricesDFMinutes=tradePricesDFMinutes[mask]
tradePricesDFMinutes=tradePricesDFMinutes.ffill(axis=0)
tradePricesDFMinutes = tradePricesDFMinutes.fillna(priceforNAVfilling)

etfprice = tradePricesDFMinutes[etfname]
tradePricesDFMinutes = tradePricesDFMinutes.pct_change().dropna() * 100

etfpricechange = tradePricesDFMinutes[etfname]
del tradePricesDFMinutes[etfname]

In [ ]:
tradePricesDFMinutes

In [ ]:
allData.quotesDataDf['Time'] = allData.quotesDataDf['Time'].apply(lambda x: helperObj.getHumanTime(ts=x, divideby=1000000000))
allData.quotesDataDf['Time'] = allData.quotesDataDf['Time'].map(lambda x: x.replace(second=0))
allData.quotesDataDf = allData.quotesDataDf[allData.quotesDataDf['Bid Size'] != 0]
allData.quotesDataDf = allData.quotesDataDf[allData.quotesDataDf['Ask Size'] != 0]
allData.quotesDataDf['Total Bid Ask Size'] = allData.quotesDataDf['Ask Size'] + allData.quotesDataDf['Bid Size']
allData.quotesDataDf['Spread'] = allData.quotesDataDf['Ask Price'] - allData.quotesDataDf['Bid Price']
allData.quotesDataDf['MidPrice'] = (allData.quotesDataDf['Ask Price'] + allData.quotesDataDf['Bid Price']) / 2
quotesSpreadsMinutes = allData.quotesDataDf.groupby('Time')['Spread'].mean()

In [ ]:
del allData

In [ ]:
netassetvaluereturn = tradePricesDFMinutes.assign(**etfData.getETFWeights()).mul(tradePricesDFMinutes).sum(axis=1)

ds = pd.concat([etfprice, etfpricechange, netassetvaluereturn, quotesSpreadsMinutes], axis=1).dropna()
ds.columns = ['ETF Price', 'ETF Change Price %', 'Net Asset Value Change%', 'ETF Trading Spread in $']
ds['Arbitrage in $'] = (ds['ETF Change Price %'] - ds['Net Asset Value Change%']) * ds['ETF Price'] / 100
ds['Arbitrage in $'] = abs(ds['Arbitrage in $'])
ds['Flag'] = 0
ds.loc[(ds['Arbitrage in $'] > ds['ETF Trading Spread in $']) & ds['ETF Trading Spread in $'] != 0, 'Flag'] = 111

holdingsChange = EtfMover(df=tradePricesDFMinutes, columnName='Change%')
etfMoverholdings = EtfMover(df=tradePricesDFMinutes.assign(**etfData.getETFWeights()).mul(tradePricesDFMinutes).dropna(axis=1), columnName='ETFMover%')

ds=pd.concat([ds,etfMoverholdings,holdingsChange],axis=1)